In [15]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
num_merges = 256

In [1]:
def merge(ids, pair, idx):
    """
    ids = list of inputs
    pair = target pair to replace
    idx = number to replace with
    """

    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2 # skip over two indices
        else:
            newids.append(ids[i])
            i += 1
    return newids

    # print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))

In [2]:


def get_stats(bytes):
    # loops through all existing byte pairs and counts them
    counts = {}
    for pair in zip(bytes, bytes[1:]):
        counts[pair] = counts.get(pair, 0) + 1

    return counts

In [3]:
vocab = { idx: bytes([idx]) for idx in range(256)} # create a dict that maps ints to byte versions


In [7]:
ids = text.encode("utf-8")
tokens = list(map(int, ids)) # convert to a list of integers in range 0..255 for convenience

In [ ]:
tokens

In [ ]:
merges = {}

for i in range(num_merges):
        stats = get_stats(tokens) # get counts of each byte pair
        pair = max(stats, key=stats.get) # find largest pair by value (number of occurences)
        idx = 256 + i # create new token value
        tokens = merge(tokens, pair, idx) # replace all instances of the pair with the new token
        merges[pair] = idx 

In [32]:
for (p0, p1), idx in merges.items():
        vocab[idx] = vocab[p0] + vocab[p1] # iterate through merges, setting the newly minted tokens' values to be the bytes of the merged pair

In [35]:

def decode(ids): # ids is a series of integers that represent tokens. 1-256 are the og tokens, 256+ are newly created ones
    tokens = b"".join(vocab[idx] for idx in ids) # basically just create a bytestream 
    text = tokens.decode("utf-8", errors="replace") # errors=replace to make sure invalid bytes get converted properly
    return text

In [36]:
def encode(text):
    # given string, return list of integers (tokens )
    tokens = list(text.encode("utf-8"))

    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf"))) # take all the pairs from stats, look in merges dict, find the lowest merged token number
        # ie 257 not 258, and return the pair that corresponds to that merge
        if pair not in merges:
            break # nothign else can be merged
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(decode(encode("bruh")))

# so we basically trained a tokenizer
# the weights manifest in terms of:
# - the vocab contains the mappings of the token id to the byte(s) that form that token
# - the merges represent how the bytes were merged to form the token ids



bruh


In [ ]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
# regex goes from left to right on the matching string ^ and tries to match. in the first example in the vid, "Hello World", the first thing that matches is
# the  ?\p{L}+, which means optional space, p{L}+ which means any number of letters from any language. in "Hello World", this will create two matches, as the
# second space does not count as a letter

# the section that is ^\s... means, match something that is not a letter, number or space, so its matching for punctuation, like ?!!?!
# the section after that is a space lookahead, trying to match !\S, meaning that if theres multiple white spaces, the regex will only match up until the second to last space
# this is important because space + word is a very common token, so we want to make more of those tokens, 

# one interesting thing to note is that the gpt2 authors note that they didn't include regex.IGNORECASE, which basically nullifies a lot of the matching
# eg: WORD'S will match as WORD, ', S instead of WORD, 'S

# also, idk how this works with nonenglish languages.

# basically what the gpt tokenizer does is it first splits the text into texts of text, and then does the tokenization/merging separately per text
# afterward, everything is concatenated. not too sure what concatenated means, im assuming it means like continue merging after the individual merges happen
print(re.findall(gpt2pat, "Hello123 World's"))

In [47]:
# testing tiktoken
import tiktoken

# gpt 2
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("<|endoftext|>", allowed_special={"<|endoftext|>"})) #idk why this no work :(


enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("    hello world!!!"))

# one interesting change in the gpt 4 tokenizer is that they do matching for 3 numbers max, which is to prevent very long numbers to become tokens
# another interesting thing that karpathy notes is that for gpt 2, there is 256 byte tokens, 50k merges, and 1 special token, in which that special one is <|endoftext|>

# karpathy notes that in the fine tuned gpt models, there are actually a bunch more tokens like <|im_start|> (imaginary monologue) start for the chat gpt interface
# fill in the middle (FIM) is another special token that they have 

[50256]
[262, 24748, 1917, 12340]
